In [2]:
import sys, json, requests
sys.path.append('../../scripts/')
from s3_support import *

In [1]:
def get_last_id():
    q = '''select max(id) as last_id from transactions_expresscheckout'''
    d = redshift_query_read(q, schema='production')
    return d['last_id'].iloc[0]

In [3]:
def fetch_transactions_df(last_id=None, pull_date=None, limit=5000):
    '''
    Pull transactions from production environment
    '''
    url = 'https://secure.qgiv.com/admin/qgivadmin/statistics/trans_export.php'
    payload = dict(key='DSQR59VwyFhw21PKDF4K', last_id=last_id, limit=limit)

    rsp = requests.post(url, data=payload)
    content = json.loads(rsp.content)

    trans_df = pd.DataFrame(content, columns=content[0].keys())
    
    trans_df['id'] = trans_df['id'].astype(int)
    trans_df['isexpresscheckout'] = trans_df['isExpressCheckout']

    return trans_df[['id', 'timestamp', 'isexpresscheckout']]

In [ ]:
# starting from 4/15/23
last_id = get_last_id()
limit = 5000
ec_data = None
counter = 0

while True:
    df = fetch_transactions_df(last_id=last_id, limit=limit)
    
    ec_data = pd.concat([ec_data, df])
    last_id = ec_data['id'].max()
    ec_data = ec_data[ec_data['isexpresscheckout']!='0']
    
    counter += 1
    if counter % 10 == 0:
        print("completed iteration {}, max date: {}".format(counter, df['timestamp'].max()))
    
    if len(df) < limit:
        break

completed iteration 10, max date: 2023-08-01 13:20:09
completed iteration 20, max date: 2023-08-03 19:29:51
completed iteration 30, max date: 2023-08-06 10:35:17
completed iteration 40, max date: 2023-08-09 08:06:15
completed iteration 50, max date: 2023-08-12 07:52:17
completed iteration 60, max date: 2023-08-17 13:38:53
completed iteration 70, max date: 2023-08-23 08:12:26
completed iteration 80, max date: 2023-08-29 21:35:05
completed iteration 90, max date: 2023-09-01 09:46:02
completed iteration 100, max date: 2023-09-06 09:36:01
completed iteration 110, max date: 2023-09-10 21:31:51


In [9]:
ec_data.tail(3)

,id,timestamp,isexpresscheckout
1392,22477454,2023-10-04 09:25:28,1
1641,22477705,2023-10-04 10:26:05,1
78,22481157,2023-10-04 16:47:24,1


In [8]:
len(ec_data)

1898

In [ ]:
save_dataframe_to_file('qgiv-stats-data', 'trans.expresscheckout.csv', ec_data)

In [29]:
q = '''create table if not exists transactions_expresscheckout (
          id int not null distkey sortkey,
          date timestamp,
          isexpresscheckout boolean);'''
#redshift_query_write(q, schema='production')

In [ ]:
q = '''copy transactions_expresscheckout
        from 's3://qgiv-stats-data/trans.expresscheckout.csv'
        iam_role 'arn:aws:iam::637885584661:role/AWSRoleForRedshift'
        emptyasnull
        blanksasnull
        fillrecord
        delimiter ','
        ignoreheader 1
        region 'us-east-1';'''
redshift_query_write(q, schema='production')

In [10]:
q = "select max(date) from transactions_expresscheckout"
redshift_query_read(q, schema='production')

,max
0,2023-10-04 16:47:24
